In [29]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import cv2

In [30]:
def imagen_cuadrada(img):
    if img.shape[0] == img.shape[1]:
        return img
    i, j = img.shape
    eje = max(img.shape)
    if eje == img.shape[0]:
        img = np.hstack((img, np.zeros((i , eje - j))))
    else:
        img = np.vstack((img, np.zeros((eje - i, j))))
    return img

In [31]:
def prisma_triangular(imagen, delta):
    img = imagen.copy()
    r = []
    Nr = []
    while delta < img.shape[0] and delta < img.shape[1]:
        area_total = 0
        for i in range(0, img.shape[0] // delta, delta):
            for j in range(0, img.shape[1] // delta, delta):
                med = (img[i, j] + img[i, j + delta] + img[i + delta, j] + img[i + delta, j + delta]) / 4
                A = np.sqrt(np.square(img[i, j] - img[i + delta, j]) + np.square(delta)) * np.sqrt(np.square(img[i + delta // 2, j] - med) + np.square(delta // 2)) // 2
                B = np.sqrt(np.square(img[i, j] - img[i, j + delta]) + np.square(delta)) * np.sqrt(np.square(img[i, j + delta // 2] - med) + np.square(delta // 2)) //2
                C = np.sqrt(np.square(img[i + delta, j] - img[i + delta, j + delta]) + np.square(delta)) * np.sqrt(np.square(img[i + delta, j + delta // 2] - med) + np.square(delta // 2)) // 2
                D = np.sqrt(np.square(img[i, j + delta] - img[i + delta, j + delta]) + np.square(delta)) * np.sqrt(np.square(img[i + delta // 2, j + delta] - med) + np.square(delta // 2)) // 2
                area_total += A + B + C + D
        Nr.append(area_total)
        r.append(delta)
        delta *= 4
    reg = linear_model.LinearRegression()
    reg.fit(np.log(np.array(r)).reshape(-1,1), np.log(np.array(Nr)))
    return 2 - reg.coef_[0]

In [32]:
def box_counting_diferencial(imagen):
    copia = imagen_cuadrada(imagen)
    M = copia.shape[0]
    r = []
    Nr = [] 
    for s in range(2, M // 2):
        r.append(s / M)
        nr = 0
        for i in range(1, M, s):
            for j in range(1, M, s):
                cuadricula = imagen_cuadrada(copia[i:min(i + s, M), j:min(j + s, M)])
                h = s * 256 / M
                nr += np.ceil(np.max(cuadricula) / h) - np.ceil(np.min(cuadricula) / h) + 1
        Nr.append(nr)
    reg = linear_model.LinearRegression()
    reg.fit(np.log(np.array(r)).reshape(-1,1), np.log(np.array(Nr)))
    return -reg.coef_[0]

In [33]:
# This cell is only for a primary implementation, it is not computationally aviable# para que fuese 
def variograma(imagen):
    img = imagen.copy()
    n = np.int64(np.sqrt(min(img.shape)))
    r = []
    Nr = []
    for h in range(2, n):
        yh = 0
        tot = 0
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                dist = [(i2, j2) for i2 in range(img.shape[0]) for j2 in range(img.shape[1]) if max(i - i2, j - j2) < h]
                tot += len(dist)
                yh += sum([np.square(img[i, j] - img[i2, j2]) for (i2, j2) in dist])
        Nr.append(yh/2*tot)
        r.append(h)
    reg = linear_model.LinearRegression()
    reg.fit(np.log(np.array(r)).reshape(-1,1), np.log(np.array(Nr)))
    return 3 - reg.coef_[0]
